In [2]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import pyautogui
import pyperclip
import pygetwindow as gw

def copiar_dados_planilha(origem, destino):
    try:
        df = pd.read_excel(origem, sheet_name='Planilha1', header=0)
        print(df.head())
    except Exception as e:
        print(f"Erro ao carregar a planilha: {e}")
        return None

    colunas_necessarias = ['Número Guia']
    if not all(coluna in df.columns for coluna in colunas_necessarias):
        print(f"Erro: uma ou mais colunas não foram encontradas na planilha 'Planilha1'.")
        return None

    df_filtrado = df[colunas_necessarias].copy()
    df_filtrado['Alteração de Status'] = ""  # Adiciona a nova coluna para status

    try:
        df_filtrado.to_excel(destino, index=False)
        print(f"Dados salvos com sucesso em: {destino}")
    except Exception as e:
        print(f"Erro ao salvar a nova planilha: {e}")

    return df_filtrado

def abrir_site_e_login(url, username, password):
    try:
        service = Service(GeckoDriverManager().install())
        global driver
        driver = webdriver.Firefox(service=service)
        driver.get(url)

        user_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "login"))
        )
        pass_field = driver.find_element(By.ID, "senha")

        user_field.send_keys(username)
        pass_field.send_keys(password)
        pass_field.send_keys(Keys.RETURN)
        
        print("Login realizado com sucesso.")
        return driver
    except Exception as e:
        print(f"Erro ao abrir o site e realizar login: {e}")
        return None

def clicar_analista_financeiro(driver):
    try:
        analista_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Analista Financeiro')]"))
        )
        analista_link.click()
        print("Clicou em 'Analista Financeiro'.")
    except Exception as e:
        print(f"Erro ao clicar em 'Analista Financeiro': {e}")

def clicar_menu_cadastros(driver):
    try:
        cadastros_menu = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "2"))
        )
        cadastros_menu.click()
        print("Clicou no menu 'Cadastros'.")

        financeiros_submenu = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Financeiros')]"))
        )
        financeiros_submenu.click()
        print("Clicou no submenu 'Financeiros'.")

        debitos_submenu = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[8]/div[3]/ul[3]/li/ul/li[2]/ul/li[2]/a"))
        )
        debitos_submenu.click()
        print("Clicou no submenu 'Financeiro - Consultar Guias'.")
    except Exception as e:
        print(f"Erro ao clicar nos menus: {e}")

def copiar_numero_guia(driver, numero_guia):
    try:
        driver.switch_to.default_content()

        # Acesse todos os iframes
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        
        for iframe in iframes:
            driver.switch_to.frame(iframe)  # Tenta acessar o iframe
            try:
                guia_input = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='numeroGuiaCompleto']"))
                )
                guia_input.clear()
                guia_input.send_keys(numero_guia)
                print(f"Colou o número do processo: {numero_guia}")
                driver.switch_to.default_content()  # Retorna ao conteúdo principal
                return
            except Exception as e:
                print(f"Elemento não encontrado nesse iframe: {e}")
                driver.switch_to.default_content()  # Retorna ao conteúdo principal para tentar o próximo iframe

        print("Número da guia não encontrado dentro dos iframes ou na página atual.")
    except Exception as e:
        print(f"Erro ao colar o número da guia: {e}")

def clicar_botao_consultar(driver):
    try:
        driver.switch_to.default_content()  # Volta ao contexto principal

        # Acesse todos os iframes
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        
        for iframe in iframes:
            driver.switch_to.frame(iframe)  # Tenta acessar o iframe
            try:
                botao_consultar = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Consultar')]"))
                )
                botao_consultar.click()
                print("Botão 'Consultar' clicado.")
                driver.switch_to.default_content()  # Retorna ao conteúdo padrão
                return
            except Exception as e:
                print(f"Erro ao tentar clicar no botão dentro do iframe: {e}")
                driver.switch_to.default_content()  # Retorna ao conteúdo principal

    except Exception as e:
        print(f"Erro ao acessar o botão 'Consultar': {e}")

def clicar_botao_editar(driver):
    try:
        driver.switch_to.default_content()  # Volta ao contexto principal

        # Acesse todos os iframes
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        
        for iframe in iframes:
            driver.switch_to.frame(iframe)  # Tenta acessar o iframe
            try:
                botao_editar = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/form/div[2]/fieldset/table/tbody/tr/td[12]/button/i"))
                )
                botao_editar.click()
                print("Clicou em 'Editar Débito'.")
                driver.switch_to.default_content()  # Retorna ao conteúdo principal
                return
            except Exception as e:
                print(f"Erro ao tentar clicar no botão dentro do iframe: {e}")
                driver.switch_to.default_content()  # Retorna ao conteúdo principal

    except Exception as e:
        print(f"Erro ao acessar o botão 'Editar': {e}")

def main():
    # Caminhos do arquivo
    caminho_arquivo = r'S:/CAJ/Servidores/Murielle/VERDE.xlsx'
    caminho_arquivo_destino = os.path.join(os.path.expanduser('~'), 'Desktop', 'Teste Python 123.xlsx')

    df_filtrado = copiar_dados_planilha(caminho_arquivo, caminho_arquivo_destino)
    if df_filtrado is None:
        return

    url = 'https://projudi.tjgo.jus.br/'
    username = '70273080105'
    password = '235791Mu@'
    
    driver = abrir_site_e_login(url, username, password)
    
    if driver is None:
        return
    
    clicar_analista_financeiro(driver)
    clicar_menu_cadastros(driver)

    # Iterar sobre os números de guia na tabela
    for index, row in df_filtrado.iterrows():
        numero_guia = row['Número Guia']

        copiar_numero_guia(driver, numero_guia)  # Copia o número da guia
        clicar_botao_consultar(driver)          # Clica no botão 'Consultar'
        clicar_botao_editar(driver)             # Clica no botão 'Editar'
        break

if __name__ == "__main__":
    main()


             Número Processo                       Nome Parte      ID  \
0  0000012.50.2020.8.09.0175  PAULO HENRIQUE DE OLIVEIRA MOTA  592804   
1  0000020.33.2020.8.09.0173     Paulo Henrique Ferreira Lima  158851   
2  0000021.18.2020.8.09.0173      WELLINGTON CARDOSO DA SILVA  288985   
3  0000048.65.2020.8.09.0087    Nivaldo Lourenço Martins Neto  531694   
4  0000078.52.2017.8.09.0137             Pedro Henrique Rocha  534770   

    Número Guia  Valor Guia  Valor Original  Juros  Multa  Total Guia  
0  5977882-2/50     3460.69             NaN    NaN    NaN         NaN  
1  3181196-5/50     1753.04             NaN    NaN    NaN         NaN  
2  4133055-2/50     2312.71             NaN    NaN    NaN         NaN  
3  4790153-5/50     3100.48             NaN    NaN    NaN         NaN  
4  5182445-0/50     5559.92             NaN    NaN    NaN         NaN  
Dados salvos com sucesso em: C:\Users\mferreirasantos\Desktop\Teste Python 123.xlsx
Login realizado com sucesso.
Erro ao clicar e

KeyboardInterrupt: 

In [3]:
def capturar_valores(driver):
    try:
        # Mude para o iframe específico
        driver.switch_to.frame(driver.find_element(By.ID, 'Principal'))

        # Obtenha o conteúdo do iframe
        iframe_content = driver.page_source
        soup = BeautifulSoup(iframe_content, 'html.parser')

        # Extração de 'Valor Originário da Guia' (dentro do <b> do 2° <td> no 1° <tr>)
        valor_originario_elem = soup.select_one(
            'div#divCorpo.divCorpo > form#GuiaPreviaCalculo > div#divEditar.divEditar > '
            'fieldset.formEdicao > fieldset.VisualizaDados > table#Tabela.Tabela > tbody > '
            'tr:nth-of-type(1) > td:nth-of-type(2) > b'
        )
        valor_originario = valor_originario_elem.get_text(strip=True) if valor_originario_elem else "Não encontrado"
        print(f"'Valor Originário da Guia': {valor_originario}")


        # Extração de 'JUROS FUNDESP-PJ S F(Reg.JURO)' (3° <td> do 2° <tr>)
        juros_fundesp_elem = soup.select_one(
            'div#divCorpo.divCorpo > form#GuiaPreviaCalculo > div#divEditar.divEditar > '
            'fieldset.formEdicao > fieldset.VisualizaDados > table#Tabela.Tabela > #tabListaEscala1> '
            'tr:nth-child(1) > td:nth-child(6)'
        )

        if not juros_fundesp_elem:
            print("Estrutura para 'JUROS FUNDESP-PJ S F(Reg.JURO)' não encontrada! Verificando HTML...")
            print(soup.prettify())  # Imprima a estrutura HTML para debug
        else:
            print(f"'JUROS FUNDESP-PJ S F(Reg.JURO)': {juros_fundesp_elem.get_text(strip=True)}")

        # Extração de 'MULTA FUNDESP/PJ(Reg.M.FUNDESP)' (3° <td> do 3° <tr>)
        multa_fundesp_elem = soup.select_one(
            'div#divCorpo.divCorpo > form#GuiaPreviaCalculo > div#divEditar.divEditar > '
            'fieldset.formEdicao > fieldset.VisualizaDados > table#Tabela.Tabela > #tabListaEscala1> '
            'tr:nth-child(2) > td:nth-child(6)'
        )
        multa_fundesp = multa_fundesp_elem.get_text(strip=True) if multa_fundesp_elem else "Não encontrado"
        print(f"'MULTA FUNDESP/PJ(Reg.M.FUNDESP)': {multa_fundesp}")

        # Extração de 'Total Guia Atualizado (24/09/2024)' (<b> no 2° <td> do 4° <tr>)
        total_guia_atualizado_elem = soup.select_one(
            'div#divCorpo.divCorpo > form#GuiaPreviaCalculo > div#divEditar.divEditar > '
            'fieldset.formEdicao > fieldset.VisualizaDados > table#Tabela.Tabela > tbody:nth-child(6) > '
            'tr:nth-of-type(1) > td:nth-of-type(2) > b'
        )
        total_guia_atualizado = total_guia_atualizado_elem.get_text(strip=True) if total_guia_atualizado_elem else "Não encontrado"
        print(f"'Total Guia Atualizado (24/09/2024)': {total_guia_atualizado}")

    except Exception as e:
        print(f"Erro durante a extração: {e}")

    finally:
        # Fechar ou sair do iframe
        driver.switch_to.default_content()

# Executar a função
capturar_valores(driver)

Erro durante a extração: Message: Unable to locate element: [id="Principal"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

